In [28]:
# TOTAL LISTINGS
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from datetime import datetime, timedelta
import calendar
import pandas as pd
from openpyxl import load_workbook
import olx_datalake

# Convertendo a data inicial e final para datetime
start_date_str = '2023-06-01'
start_date = pd.to_datetime(start_date_str)

# Converte as datas para objetos datetime
start_date = datetime.strptime(start_date_str, '%Y-%m-%d')
start_date = start_date.strftime('%Y-%m-%d')
#conversoes necessarias:
year_str, month_str, day_str = start_date.split('-')

# Convert year_str and month_str to integers
year = int(year_str)
month = int(month_str)

#chamada do datalake_olx
dl = olx_datalake.DataLake()


print("Olx_datalake configurado")

#Total listings-olx
total_olx = dl.sql("""
    with 

cl as (
SELECT event_date 
FROM ods.calendar
where cast(array_join(array[year(event_date), month(event_date), day(event_date)], '-') as date) between date('"""+start_date+"""') and date('"""+start_date+"""')+ interval '4' month

)

,listings AS 
(
SELECT DISTINCT
    CAST(cl.event_date AS DATE) AS data,
    a.list_id_nk,
    ad.account_id_fk
FROM olx_listing.ad_inventory a
JOIN  cl ON DATE (a.start_date) <= DATE(cl.event_date) AND coalesce(DATE(a.end_date),current_date) >= DATE(cl.event_date)
left join ods.ad ad on a.list_id_nk = ad.list_id_nk
LEFT JOIN ods.dm_category ca ON ad.category_id_fk=ca.category_id_pk 
WHERE category_id_fk in (121, 124, 125, 2, 3, 79, 44, 86, 80, 40)
)

,daily_ads as (
SELECT 
    data,
    COUNT(distinct list_id_nk) AS listings
FROM listings
GROUP BY 1
)

select 
  date_trunc('month', data) as publi_month,
  avg(listings) as olx
from daily_ads
group by 1
order by 1
""")

#Total listings-Shift
total_shift = dl.sql("""
    with

base as (
select
count(distinct id) as listings,
dt as publi_date,
date_trunc('month',dt) as publi_month,
case when link like '%casamineira.com%' then 'Casa Mineira'
     when link like '%loft.com%' then 'Loft'
     when link like '%imovelweb.com%' then 'Imovelweb'
     when link like '%quintoandar.com%' then 'Quinto Andar' end as player
from strategy.shift
where dt >= date('"""+start_date+"""')
group by 2,3,4
),

mensal as (
select
publi_month,
player,
avg(listings) as listings
from base
group by 1,2
)
select
a.publi_month,
a.listings as quintoandar,
b.listings as loft,
c.listings as imovelweb,
d.listings as casamineira
from (select * from mensal where player = 'Quinto Andar') as a
left join (select * from mensal where player = 'Loft') as b on a.publi_month=b.publi_month
left join (select * from mensal where player = 'Imovelweb') as c on a.publi_month=c.publi_month
left join (select * from mensal where player = 'Casa Mineira') as d on a.publi_month=d.publi_month
order by 1

""")

#Total listings-zap/vr
total_zap = dl.sql("""
    with
aux as (
select dt,
       date_trunc('month',dt) as mes,
       count(distinct id) as n_listing
        from silver_normalized_listings.listings_daily
        where True
        and dt >=date('"""+start_date+"""')
        and is_active = true
        and is_zap_portal = true
        
        group by 1,2--,2,3,4,5)
        ),
   final as (     
        select
        mes as publi_month,
        avg(n_listing) as zap
        from aux
        group by 1
        )
select * from final
order by publi_month
""")

total_vr = dl.sql("""
    with
aux as (
select dt,
       date_trunc('month',dt) as mes,
       count(distinct id) as n_listing
        from silver_normalized_listings.listings_daily
        where True
        and dt >=date('"""+start_date+"""')
        and is_active = true
        and is_vr_portal = true
        
        group by 1,2--,2,3,4,5)
        ),
   final as (     
        select
        mes as publi_month,
        avg(n_listing) as vivareal
        from aux
        group by 1
        )
select * from final
order by publi_month

""")

result = total_shift.merge(total_zap, on='publi_month').merge(total_vr, on='publi_month').merge(total_olx, on='publi_month')

# Exibindo o resultado
print(result_total)


Olx_datalake configurado


FINISHED: 100%|██████████| 1574/1574 [01:35<00:00, 16.55 splits/s]

  publi_month  quintoandar      loft  imovelweb  casamineira           zap  \
0  2023-06-01     215941.4  91472.40  3712163.4    1965700.6  6.891894e+06   
1  2023-07-01     223527.0  93198.25  3419960.2    1833958.5  6.993771e+06   
2  2023-08-01     228584.5  94614.25  3646638.0    1715907.5  7.159903e+06   
3  2023-09-01     228532.5  96913.50  3880641.0    1623684.0  7.126489e+06   

       vivareal           olx  
0  6.892801e+06  7.940828e+06  
1  6.994696e+06  8.031011e+06  
2  7.161266e+06  8.150936e+06  
3  7.127475e+06  8.159988e+06  


In [30]:
#total listings WRITE
from datetime import datetime
import calendar

def encontrar_coluna_por_data(credenciais_json, nome_planilha, nome_aba, data_procurada):

    # Abra a planilha e a aba
    planilha = client.open(nome_planilha)
    aba = planilha.worksheet(nome_aba)

    # Obtenha os valores da linha 10 (assumindo que as datas estão na linha 10)
    valores_linha_10 = aba.row_values(10)

    # Itere pelos valores para encontrar a coluna com a data procurada
    for coluna, data in enumerate(valores_linha_10, start=1):
        if data == data_procurada:
            return coluna

    return None  # Retorna None se a data não for encontrada

def converter_data(mon_yy):
    # Converte a data no formato "Mon/YY" para "01/MM/YY"
    mon_yy = "01/" + mon_yy
    # Converte a data para o formato "YYYY-MM-01 00:00:00"
    data_formatada = pd.to_datetime(mon_yy, format="%d/%b/%y")
    return data_formatada

def dataf_max_date(df):
    return df['publi_month'].max()

def column_number(col_str):
    # Converte uma string de coluna em seu número inteiro correspondente (A=1, B=2, ..., Z=26, AA=27, AB=28, ...)
    num = 0
    for c in col_str:
        num = num * 26 + ord(c) - ord('A') + 1
    return num

def column_string(column_num):
    # Converte um número de coluna em sua representação de string correspondente (1=A, 2=B, ..., 26=Z, 27=AA, 28=AB, ...)
    result = ""
    while column_num > 0:
        column_num, remainder = divmod(column_num - 1, 26)
        result = chr(ord('A') + remainder) + result
    return result


def preencher_datas_na_planilha(credenciais_json, nome_planilha, nome_aba, df):
    # Carregue as credenciais do Google Sheets
    scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
    creds = ServiceAccountCredentials.from_json_keyfile_name(credenciais_json, scope)
    client = gspread.authorize(creds)

    # Abra a planilha e a aba
    planilha = client.open(nome_planilha)
    aba = planilha.worksheet(nome_aba)

    # Encontre a maior data no dataframe
    data_maxima_dataframe = dataf_max_date(df)
    # Comece da primeira célula na linha 10
    coluna_atual = 'AM'
    celula_atual = aba.acell(f'{coluna_atual}10').value
    celula_atual=converter_data(celula_atual)
    while celula_atual<data_maxima_dataframe:
        # Obtenha o valor da célula atual e da próxima célula
        print('loop iter')
        celula_atual = aba.acell(f'{coluna_atual}10').value
        celula_atual=converter_data(celula_atual)
        coluna_proxima_numero = column_number(coluna_atual) + 1
        coluna_proxima = column_string(coluna_proxima_numero)
        celula_proxima = aba.acell(f'{coluna_proxima}10').value

        # Verifique se a próxima célula está preenchida
        if not celula_proxima:
            # Se a célula atual estiver vazia, preencha com a data um mês à frente
            nova_data =celula_atual+ timedelta(days=31)
            nova_data = nova_data.replace(day=1)  # Define o dia para 1 para garantir que seja o primeiro dia do mês
            data_atual_str = nova_data.strftime('%b/%y')
            aba.update_acell(f'{coluna_proxima}10', data_atual_str)
            # Avance para a próxima coluna
        coluna_atual = coluna_proxima
            

    print('Datas atualizadas com sucesso no Google Sheets.')

# Defina o nome do seu arquivo JSON de credenciais do Google
credenciais_json = 'project-bht-e1805246c668.json'

# Defina o nome do arquivo do Google Sheets e a aba
nome_planilha = 'Q2-23 Competition KPI collection file - Brasil'
nome_aba = '(datalake) Listings RE'

# Carregue as credenciais do Google Sheets
scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
creds = ServiceAccountCredentials.from_json_keyfile_name(credenciais_json, scope)
client = gspread.authorize(creds)

# Abra a planilha e a aba
planilha = client.open(nome_planilha)
aba = planilha.worksheet(nome_aba)

# Carregue o dataframe
df = result_total  
preencher_datas_na_planilha(credenciais_json, nome_planilha, nome_aba, df)

# Preencha as células com os dados do dataframe
colunas_players = df.columns[df.columns != 'publi_month']  # Obtém as colunas dos players
for index, row in df.iterrows():
    data = row['publi_month'].strftime('%b/%y')
    print(data)

    # Encontre a coluna correspondente à data
    coluna_data =encontrar_coluna_por_data(credenciais_json, nome_planilha, nome_aba, data)

    # Atualize as células com os valores dos players
    for player in colunas_players:
        valor = row[player]
        coluna_a = aba.col_values(1)
        coluna_a=coluna_a[10:17]
        print(coluna_a)
        for linha, site in enumerate(coluna_a, start=11):
            if site.lower() == player.lower():
                linha_player=linha
                print(linha_player)
        aba.update_acell(f'{column_string(coluna_data)}{linha_player}', valor)

print('Dados atualizados com sucesso no Google Sheets.')



loop iter
loop iter
loop iter
loop iter
loop iter
loop iter
loop iter
loop iter
loop iter
Datas atualizadas com sucesso no Google Sheets.
Jun/23
['casamineira', 'imovelweb', 'loft', 'olx', 'quintoandar', 'vivareal', 'zap']
15
['casamineira', 'imovelweb', 'loft', 'olx', 'quintoandar', 'vivareal', 'zap']
13
['casamineira', 'imovelweb', 'loft', 'olx', 'quintoandar', 'vivareal', 'zap']
12
['casamineira', 'imovelweb', 'loft', 'olx', 'quintoandar', 'vivareal', 'zap']
11
['casamineira', 'imovelweb', 'loft', 'olx', 'quintoandar', 'vivareal', 'zap']
17
['casamineira', 'imovelweb', 'loft', 'olx', 'quintoandar', 'vivareal', 'zap']
16
['casamineira', 'imovelweb', 'loft', 'olx', 'quintoandar', 'vivareal', 'zap']
14
Jul/23
['casamineira', 'imovelweb', 'loft', 'olx', 'quintoandar', 'vivareal', 'zap']
15
['casamineira', 'imovelweb', 'loft', 'olx', 'quintoandar', 'vivareal', 'zap']
13
['casamineira', 'imovelweb', 'loft', 'olx', 'quintoandar', 'vivareal', 'zap']
12
['casamineira', 'imovelweb', 'loft', '

In [31]:
# SALE LISTINGS
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from datetime import datetime, timedelta
import calendar
import pandas as pd
from openpyxl import load_workbook
import olx_datalake

# Convertendo a data inicial e final para datetime
start_date_str = '2023-06-01'
start_date = pd.to_datetime(start_date_str)

# Converte as datas para objetos datetime
start_date = datetime.strptime(start_date_str, '%Y-%m-%d')
start_date = start_date.strftime('%Y-%m-%d')
#conversoes necessarias:
year_str, month_str, day_str = start_date.split('-')

# Convert year_str and month_str to integers
year = int(year_str)
month = int(month_str)

#chamada do datalake_olx
dl = olx_datalake.DataLake()


print("Olx_datalake configurado")

#Sale listings-olx
sale_olx = dl.sql("""
    with 

cl as (
SELECT event_date 
FROM ods.calendar
where cast(array_join(array[year(event_date), month(event_date), day(event_date)], '-') as date) between date('"""+start_date+"""') and date('"""+start_date+"""')+ interval '4' month

)

,listings AS 
(
SELECT DISTINCT
    CAST(cl.event_date AS DATE) AS data,
    a.list_id_nk,
    ad.account_id_fk,
    case when ad_type_id_fk=1 then 'venda'
    when ad_type_id_fk=3 then 'aluguel' end as negocio
FROM olx_listing.ad_inventory a
JOIN  cl 
    ON DATE (a.start_date) <= DATE(cl.event_date) AND coalesce(DATE(a.end_date),current_date) >= DATE(cl.event_date)
left join ods.ad ad on a.list_id_nk = ad.list_id_nk
LEFT JOIN ods.dm_category ca ON ad.category_id_fk=ca.category_id_pk 
left join ods.dm_ad_type as neg on neg.ad_type_id_pk = ad.ad_type_id_fk
WHERE category_id_fk in (121, 124, 125, 2, 3, 79, 44, 86, 80, 40)
)

,daily_ads as (
SELECT 
    data,
    COUNT(distinct list_id_nk) AS listings
FROM listings
where negocio='venda'
GROUP BY 1
)

select 
  date_trunc('month', data) as publi_month,
  avg(listings) as olx
from daily_ads
group by 1
order by 1
""")

#Sale listings-Shift
sale_shift = dl.sql("""
    with

base as (
select
count(distinct id) as listings,
dt as publi_date,
date_trunc('month',dt) as publi_month,
case when link like '%casamineira.com%' then 'Casa Mineira'
     when link like '%loft.com%' then 'Loft'
     when link like '%imovelweb.com%' then 'Imovelweb'
     when link like '%quintoandar.com%' then 'Quinto Andar' end as player
from strategy.shift
where dt >= date('"""+start_date+"""')
and tipo in (1,2) -- venda
group by 2,3,4
),

mensal as (
select
publi_month,
player,
avg(listings) as listings
from base
group by 1,2
)
select
a.publi_month,
a.listings as quintoandar,
b.listings as loft,
c.listings as imovelweb,
d.listings as casamineira
from (select * from mensal where player = 'Quinto Andar') as a
left join (select * from mensal where player = 'Loft') as b on a.publi_month=b.publi_month
left join (select * from mensal where player = 'Imovelweb') as c on a.publi_month=c.publi_month
left join (select * from mensal where player = 'Casa Mineira') as d on a.publi_month=d.publi_month
order by 1

""")

#sale listings-zap/vr
sale_zap = dl.sql("""
    with
aux as (
select dt,
       date_trunc('month',dt) as mes,
       count(distinct id) as n_listing
        from silver_normalized_listings.listings_daily
        where True
        and dt >=date('"""+start_date+"""')
        and is_active = true
        and is_zap_portal = true
        and is_sale = true
        
        group by 1,2--,2,3,4,5)
        ),
   final as (     
        select
        mes as publi_month,
        avg(n_listing) as zap
        from aux
        group by 1
        )
select * from final
order by publi_month
""")

sale_vr = dl.sql("""
    with
aux as (
select dt,
       date_trunc('month',dt) as mes,
       count(distinct id) as n_listing
        from silver_normalized_listings.listings_daily
        where True
        and dt >=date('"""+start_date+"""')
        and is_active = true
        and is_vr_portal = true
        and is_sale = true
        
        group by 1,2--,2,3,4,5)
        ),
   final as (     
        select
        mes as publi_month,
        avg(n_listing) as vivareal
        from aux
        group by 1
        )
select * from final
order by publi_month

""")

result_sale = sale_shift.merge(sale_zap, on='publi_month').merge(sale_vr, on='publi_month').merge(sale_olx, on='publi_month')

# Exibindo o resultado
print(result_sale)

Olx_datalake configurado


FINISHED: 100%|██████████| 2690/2690 [00:33<00:00, 81.47 splits/s]

  publi_month  quintoandar      loft   imovelweb  casamineira           zap  \
0  2023-06-01    190023.60  91472.40  3379890.60   1776472.40  6.071932e+06   
1  2023-07-01    195828.25  93198.25  3064799.00   1651812.75  6.156234e+06   
2  2023-08-01    200741.75  94614.25  3379743.25   1547933.50  6.297289e+06   
3  2023-09-01    200034.00  96913.50  3604521.50   1468791.50  6.256694e+06   

       vivareal           olx  
0  6.071003e+06  6.795308e+06  
1  6.155364e+06  6.864544e+06  
2  6.297417e+06  6.961385e+06  
3  6.257130e+06  6.959081e+06  


In [32]:
#sale listings WRITE
from datetime import datetime
import calendar

def encontrar_coluna_por_data(credenciais_json, nome_planilha, nome_aba, data_procurada):

    # Abra a planilha e a aba
    planilha = client.open(nome_planilha)
    aba = planilha.worksheet(nome_aba)

    # Obtenha os valores da linha 10 (assumindo que as datas estão na linha 10)
    valores_linha_10 = aba.row_values(10)

    # Itere pelos valores para encontrar a coluna com a data procurada
    for coluna, data in enumerate(valores_linha_10, start=1):
        if data == data_procurada:
            return coluna

    return None  # Retorna None se a data não for encontrada

def converter_data(mon_yy):
    # Converte a data no formato "Mon/YY" para "01/MM/YY"
    mon_yy = "01/" + mon_yy
    # Converte a data para o formato "YYYY-MM-01 00:00:00"
    data_formatada = pd.to_datetime(mon_yy, format="%d/%b/%y")
    return data_formatada

def dataf_max_date(df):
    return df['publi_month'].max()

def column_number(col_str):
    # Converte uma string de coluna em seu número inteiro correspondente (A=1, B=2, ..., Z=26, AA=27, AB=28, ...)
    num = 0
    for c in col_str:
        num = num * 26 + ord(c) - ord('A') + 1
    return num

def column_string(column_num):
    # Converte um número de coluna em sua representação de string correspondente (1=A, 2=B, ..., 26=Z, 27=AA, 28=AB, ...)
    result = ""
    while column_num > 0:
        column_num, remainder = divmod(column_num - 1, 26)
        result = chr(ord('A') + remainder) + result
    return result


def preencher_datas_na_planilha(credenciais_json, nome_planilha, nome_aba, df):
    # Carregue as credenciais do Google Sheets
    scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
    creds = ServiceAccountCredentials.from_json_keyfile_name(credenciais_json, scope)
    client = gspread.authorize(creds)

    # Abra a planilha e a aba
    planilha = client.open(nome_planilha)
    aba = planilha.worksheet(nome_aba)

    # Encontre a maior data no dataframe
    data_maxima_dataframe = dataf_max_date(df)
    # Comece da primeira célula na linha 10
    coluna_atual = 'AM'
    celula_atual = aba.acell(f'{coluna_atual}10').value
    celula_atual=converter_data(celula_atual)
    while celula_atual<data_maxima_dataframe:
        # Obtenha o valor da célula atual e da próxima célula
        print('loop iter')
        celula_atual = aba.acell(f'{coluna_atual}10').value
        celula_atual=converter_data(celula_atual)
        coluna_proxima_numero = column_number(coluna_atual) + 1
        coluna_proxima = column_string(coluna_proxima_numero)
        celula_proxima = aba.acell(f'{coluna_proxima}10').value

        # Verifique se a próxima célula está preenchida
        if not celula_proxima:
            # Se a célula atual estiver vazia, preencha com a data um mês à frente
            nova_data =celula_atual+ timedelta(days=31)
            nova_data = nova_data.replace(day=1)  # Define o dia para 1 para garantir que seja o primeiro dia do mês
            data_atual_str = nova_data.strftime('%b/%y')
            aba.update_acell(f'{coluna_proxima}10', data_atual_str)
            # Avance para a próxima coluna
        coluna_atual = coluna_proxima
            

    print('Datas atualizadas com sucesso no Google Sheets.')

# Defina o nome do seu arquivo JSON de credenciais do Google
credenciais_json = 'project-bht-e1805246c668.json'

# Defina o nome do arquivo do Google Sheets e a aba
nome_planilha = 'Q2-23 Competition KPI collection file - Brasil'
nome_aba = '(datalake) Listings RE'

# Carregue as credenciais do Google Sheets
scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
creds = ServiceAccountCredentials.from_json_keyfile_name(credenciais_json, scope)
client = gspread.authorize(creds)

# Abra a planilha e a aba
planilha = client.open(nome_planilha)
aba = planilha.worksheet(nome_aba)

# Carregue o dataframe
df = result_sale  
preencher_datas_na_planilha(credenciais_json, nome_planilha, nome_aba, df)

# Preencha as células com os dados do dataframe
colunas_players = df.columns[df.columns != 'publi_month']  # Obtém as colunas dos players
for index, row in df.iterrows():
    data = row['publi_month'].strftime('%b/%y')
    print(data)

    # Encontre a coluna correspondente à data
    coluna_data =encontrar_coluna_por_data(credenciais_json, nome_planilha, nome_aba, data)

    # Atualize as células com os valores dos players
    for player in colunas_players:
        valor = row[player]
        coluna_a = aba.col_values(1)
        coluna_a=coluna_a[18:25]
        print(coluna_a)
        for linha, site in enumerate(coluna_a, start=19):
            if site.lower() == player.lower():
                linha_player=linha
                print(linha_player)
        aba.update_acell(f'{column_string(coluna_data)}{linha_player}', valor)

print('Dados atualizados com sucesso no Google Sheets.')

loop iter
loop iter
loop iter
loop iter
loop iter
loop iter
loop iter
loop iter
loop iter
Datas atualizadas com sucesso no Google Sheets.
Jun/23
['CasaMineira', 'ImovelWeb', 'Loft', 'OLX', 'QuintoAndar', 'VivaReal', 'ZAP']
23
['CasaMineira', 'ImovelWeb', 'Loft', 'OLX', 'QuintoAndar', 'VivaReal', 'ZAP']
21
['CasaMineira', 'ImovelWeb', 'Loft', 'OLX', 'QuintoAndar', 'VivaReal', 'ZAP']
20
['CasaMineira', 'ImovelWeb', 'Loft', 'OLX', 'QuintoAndar', 'VivaReal', 'ZAP']
19
['CasaMineira', 'ImovelWeb', 'Loft', 'OLX', 'QuintoAndar', 'VivaReal', 'ZAP']
25
['CasaMineira', 'ImovelWeb', 'Loft', 'OLX', 'QuintoAndar', 'VivaReal', 'ZAP']
24
['CasaMineira', 'ImovelWeb', 'Loft', 'OLX', 'QuintoAndar', 'VivaReal', 'ZAP']
22
Jul/23
['CasaMineira', 'ImovelWeb', 'Loft', 'OLX', 'QuintoAndar', 'VivaReal', 'ZAP']
23
['CasaMineira', 'ImovelWeb', 'Loft', 'OLX', 'QuintoAndar', 'VivaReal', 'ZAP']
21
['CasaMineira', 'ImovelWeb', 'Loft', 'OLX', 'QuintoAndar', 'VivaReal', 'ZAP']
20
['CasaMineira', 'ImovelWeb', 'Loft', '

In [33]:
# RENTAL LISTINGS
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from datetime import datetime, timedelta
import calendar
import pandas as pd
from openpyxl import load_workbook
import olx_datalake

# Convertendo a data inicial e final para datetime
start_date_str = '2023-06-01'
start_date = pd.to_datetime(start_date_str)

# Converte as datas para objetos datetime
start_date = datetime.strptime(start_date_str, '%Y-%m-%d')
start_date = start_date.strftime('%Y-%m-%d')
#conversoes necessarias:
year_str, month_str, day_str = start_date.split('-')

# Convert year_str and month_str to integers
year = int(year_str)
month = int(month_str)

#chamada do datalake_olx
dl = olx_datalake.DataLake()


print("Olx_datalake configurado")

#Rental listings-olx
rental_olx = dl.sql("""
    with 

cl as (
SELECT event_date 
FROM ods.calendar
where cast(array_join(array[year(event_date), month(event_date), day(event_date)], '-') as date) between date('"""+start_date+"""') and date('"""+start_date+"""')+ interval '4' month

)

,listings AS 
(
SELECT DISTINCT
    CAST(cl.event_date AS DATE) AS data,
    a.list_id_nk,
    ad.account_id_fk,
    case when ad_type_id_fk=1 then 'venda'
    when ad_type_id_fk=3 then 'aluguel' end as negocio
FROM olx_listing.ad_inventory a
JOIN  cl 
    ON DATE (a.start_date) <= DATE(cl.event_date) AND coalesce(DATE(a.end_date),current_date) >= DATE(cl.event_date)
left join ods.ad ad on a.list_id_nk = ad.list_id_nk
LEFT JOIN ods.dm_category ca ON ad.category_id_fk=ca.category_id_pk 
left join ods.dm_ad_type as neg on neg.ad_type_id_pk = ad.ad_type_id_fk
WHERE category_id_fk in (121, 124, 125, 2, 3, 79, 44, 86, 80, 40)
)

,daily_ads as (
SELECT 
    data,
    COUNT(distinct list_id_nk) AS listings
FROM listings
where negocio='aluguel'
GROUP BY 1
)

select 
  date_trunc('month', data) as publi_month,
  avg(listings) as olx
from daily_ads
group by 1
order by 1
""")

#Rental listings-Shift
rental_shift = dl.sql("""
    with

base as (
select
count(distinct id) as listings,
dt as publi_date,
date_trunc('month',dt) as publi_month,
case when link like '%casamineira.com%' then 'Casa Mineira'
     when link like '%loft.com%' then 'Loft'
     when link like '%imovelweb.com%' then 'Imovelweb'
     when link like '%quintoandar.com%' then 'Quinto Andar' end as player
from strategy.shift
where dt >= date('"""+start_date+"""')
and tipo in (0,2) -- aluguel
group by 2,3,4
),

mensal as (
select
publi_month,
player,
avg(listings) as listings
from base
group by 1,2
)
select
a.publi_month,
a.listings as quintoandar,
b.listings as loft,
c.listings as imovelweb,
d.listings as casamineira
from (select * from mensal where player = 'Quinto Andar') as a
left join (select * from mensal where player = 'Loft') as b on a.publi_month=b.publi_month
left join (select * from mensal where player = 'Imovelweb') as c on a.publi_month=c.publi_month
left join (select * from mensal where player = 'Casa Mineira') as d on a.publi_month=d.publi_month
order by 1

""")

#Rental listings-zap/vr
rental_zap = dl.sql("""
    with
aux as (
select dt,
       date_trunc('month',dt) as mes,
       count(distinct id) as n_listing
        from silver_normalized_listings.listings_daily
        where True
        and dt >=date('"""+start_date+"""')
        and is_active = true
        and is_zap_portal = true
         and is_rental = true
        
        group by 1,2--,2,3,4,5)
        ),
   final as (     
        select
        mes as publi_month,
        avg(n_listing) as zap
        from aux
        group by 1
        )
select * from final
order by publi_month
""")

rental_vr = dl.sql("""
    with
aux as (
select dt,
       date_trunc('month',dt) as mes,
       count(distinct id) as n_listing
        from silver_normalized_listings.listings_daily
        where True
        and dt >=date('"""+start_date+"""')
        and is_active = true
        and is_vr_portal = true
         and is_rental = true
        
        group by 1,2--,2,3,4,5)
        ),
   final as (     
        select
        mes as publi_month,
        avg(n_listing) as vivareal
        from aux
        group by 1
        )
select * from final
order by publi_month

""")

result_rental = rental_shift.merge(rental_zap, on='publi_month').merge(rental_vr, on='publi_month').merge(rental_olx, on='publi_month')

# Exibindo o resultado
print(result_rental)

Olx_datalake configurado


FINISHED: 100%|██████████| 2802/2802 [00:32<00:00, 86.80 splits/s]

  publi_month  quintoandar  loft  imovelweb  casamineira           zap  \
0  2023-06-01    107365.60   NaN   430427.8    189228.20  9.852658e+05   
1  2023-07-01    110412.75   NaN   464267.0    182145.75  1.008030e+06   
2  2023-08-01    112375.75   NaN   359731.5    167974.00  1.038758e+06   
3  2023-09-01    114703.00   NaN   377866.5    154892.50  1.045300e+06   

       vivareal           olx  
0  9.871059e+05  1.145520e+06  
1  1.009830e+06  1.166467e+06  
2  1.039999e+06  1.189551e+06  
3  1.045857e+06  1.200908e+06  


In [36]:
#Rental listings WRITE
from datetime import datetime
import calendar

def encontrar_coluna_por_data(credenciais_json, nome_planilha, nome_aba, data_procurada):

    # Abra a planilha e a aba
    planilha = client.open(nome_planilha)
    aba = planilha.worksheet(nome_aba)

    # Obtenha os valores da linha 10 (assumindo que as datas estão na linha 10)
    valores_linha_10 = aba.row_values(10)

    # Itere pelos valores para encontrar a coluna com a data procurada
    for coluna, data in enumerate(valores_linha_10, start=1):
        if data == data_procurada:
            return coluna

    return None  # Retorna None se a data não for encontrada

def converter_data(mon_yy):
    # Converte a data no formato "Mon/YY" para "01/MM/YY"
    mon_yy = "01/" + mon_yy
    # Converte a data para o formato "YYYY-MM-01 00:00:00"
    data_formatada = pd.to_datetime(mon_yy, format="%d/%b/%y")
    return data_formatada

def dataf_max_date(df):
    return df['publi_month'].max()

def column_number(col_str):
    # Converte uma string de coluna em seu número inteiro correspondente (A=1, B=2, ..., Z=26, AA=27, AB=28, ...)
    num = 0
    for c in col_str:
        num = num * 26 + ord(c) - ord('A') + 1
    return num

def column_string(column_num):
    # Converte um número de coluna em sua representação de string correspondente (1=A, 2=B, ..., 26=Z, 27=AA, 28=AB, ...)
    result = ""
    while column_num > 0:
        column_num, remainder = divmod(column_num - 1, 26)
        result = chr(ord('A') + remainder) + result
    return result

def substituir_nan_por_zero(df):
    # Substitui todos os valores NaN por zero no DataFrame
    df_sem_nan = df.fillna(0)
    return df_sem_nan

def preencher_datas_na_planilha(credenciais_json, nome_planilha, nome_aba, df):
    # Carregue as credenciais do Google Sheets
    scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
    creds = ServiceAccountCredentials.from_json_keyfile_name(credenciais_json, scope)
    client = gspread.authorize(creds)

    # Abra a planilha e a aba
    planilha = client.open(nome_planilha)
    aba = planilha.worksheet(nome_aba)

    # Encontre a maior data no dataframe
    data_maxima_dataframe = dataf_max_date(df)
    # Comece da primeira célula na linha 10
    coluna_atual = 'AM'
    celula_atual = aba.acell(f'{coluna_atual}10').value
    celula_atual=converter_data(celula_atual)
    while celula_atual<data_maxima_dataframe:
        # Obtenha o valor da célula atual e da próxima célula
        print('loop iter')
        celula_atual = aba.acell(f'{coluna_atual}10').value
        celula_atual=converter_data(celula_atual)
        coluna_proxima_numero = column_number(coluna_atual) + 1
        coluna_proxima = column_string(coluna_proxima_numero)
        celula_proxima = aba.acell(f'{coluna_proxima}10').value

        # Verifique se a próxima célula está preenchida
        if not celula_proxima:
            # Se a célula atual estiver vazia, preencha com a data um mês à frente
            nova_data =celula_atual+ timedelta(days=31)
            nova_data = nova_data.replace(day=1)  # Define o dia para 1 para garantir que seja o primeiro dia do mês
            data_atual_str = nova_data.strftime('%b/%y')
            aba.update_acell(f'{coluna_proxima}10', data_atual_str)
            # Avance para a próxima coluna
        coluna_atual = coluna_proxima
            

    print('Datas atualizadas com sucesso no Google Sheets.')

# Defina o nome do seu arquivo JSON de credenciais do Google
credenciais_json = 'project-bht-e1805246c668.json'

# Defina o nome do arquivo do Google Sheets e a aba
nome_planilha = 'Q2-23 Competition KPI collection file - Brasil'
nome_aba = '(datalake) Listings RE'

# Carregue as credenciais do Google Sheets
scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
creds = ServiceAccountCredentials.from_json_keyfile_name(credenciais_json, scope)
client = gspread.authorize(creds)

# Abra a planilha e a aba
planilha = client.open(nome_planilha)
aba = planilha.worksheet(nome_aba)

# Carregue o dataframe
df = substituir_nan_por_zero(result_rental)  
preencher_datas_na_planilha(credenciais_json, nome_planilha, nome_aba, df)

# Preencha as células com os dados do dataframe
colunas_players = df.columns[df.columns != 'publi_month']  # Obtém as colunas dos players
for index, row in df.iterrows():
    data = row['publi_month'].strftime('%b/%y')
    print(data)

    # Encontre a coluna correspondente à data
    coluna_data =encontrar_coluna_por_data(credenciais_json, nome_planilha, nome_aba, data)

    # Atualize as células com os valores dos players
    for player in colunas_players:
        valor = row[player]
        coluna_a = aba.col_values(1)
        coluna_a=coluna_a[26:33]
        print(coluna_a)
        for linha, site in enumerate(coluna_a, start=27):
            if site.lower() == player.lower():
                linha_player=linha
                print(linha_player)
        aba.update_acell(f'{column_string(coluna_data)}{linha_player}', valor)

print('Dados atualizados com sucesso no Google Sheets.')

loop iter
loop iter
loop iter
loop iter
loop iter
loop iter
loop iter
loop iter
loop iter
Datas atualizadas com sucesso no Google Sheets.
Jun/23
['CasaMineira', 'ImovelWeb', 'Loft', 'OLX', 'QuintoAndar', 'VivaReal', 'ZAP']
31
['CasaMineira', 'ImovelWeb', 'Loft', 'OLX', 'QuintoAndar', 'VivaReal', 'ZAP']
29
['CasaMineira', 'ImovelWeb', 'Loft', 'OLX', 'QuintoAndar', 'VivaReal', 'ZAP']
28
['CasaMineira', 'ImovelWeb', 'Loft', 'OLX', 'QuintoAndar', 'VivaReal', 'ZAP']
27
['CasaMineira', 'ImovelWeb', 'Loft', 'OLX', 'QuintoAndar', 'VivaReal', 'ZAP']
33
['CasaMineira', 'ImovelWeb', 'Loft', 'OLX', 'QuintoAndar', 'VivaReal', 'ZAP']
32
['CasaMineira', 'ImovelWeb', 'Loft', 'OLX', 'QuintoAndar', 'VivaReal', 'ZAP']
30
Jul/23
['CasaMineira', 'ImovelWeb', 'Loft', 'OLX', 'QuintoAndar', 'VivaReal', 'ZAP']
31
['CasaMineira', 'ImovelWeb', 'Loft', 'OLX', 'QuintoAndar', 'VivaReal', 'ZAP']
29
['CasaMineira', 'ImovelWeb', 'Loft', 'OLX', 'QuintoAndar', 'VivaReal', 'ZAP']
28
['CasaMineira', 'ImovelWeb', 'Loft', '